### Hypothetical Document Embeddings (HyDE):

* It is a retrieval technique where, instead of embedding the user's query directly you first generatea hypothetical answer(document) to the query using an LLM and then embed that hypothetical document to serach your vector store.

HyDE bridges the gap between user intent and relevant content, especially when:
1. queries are short.
2. language mismatch between query and documents.
3. you want to retrieve based on answer content, not question words.

